In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.realpath('..'))

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import plotly.express as ex
import plotly.graph_objects as go

from copy import deepcopy

import tracking_v2.callback as cb
from tracking_v2.target import NearConstantVelocityTarget
from tracking_v2.tracker import MultiTargetTracker
from tracking_v2.kalman import linear_ncv
from tracking_v2.sensor import GeometricSensor
from tracking_v2.evaluation import TrackerRunner
import tracking_v2.evaluation.runner as runner

from tracking_v2.util import to_df, display

In [3]:
display.as_png = False

In [11]:
class MyCallback:
    @cb.mht_decisions
    def mht_decisions(self, tracker, decisions):
        print([f"{d[0]}: {d[1]}" for d in decisions])
        pass
    
    @cb.associated_to_track
    def associated_to_track(self, tracker, llr, track, measurement):
        print('track updated', track.track_id, llr)

    @cb.initialized_new_track
    def initialized_new_track(self, tracker, llr, track, measurement0, measurement1):
        print('track initialized', track.track_id, llr)

    @cb.measurement_not_associated
    def measurement_not_associated(self, tracker, llr, measurement):
        #print('measurement_not_associated')
        pass

    @cb.track_not_updated
    def track_not_updated(self, tracker, llr, track):
        #print('track_not_updated')
        pass

In [12]:
target = NearConstantVelocityTarget()
sensor = GeometricSensor()

mtt = MultiTargetTracker()
runner = TrackerRunner([target], [sensor], mtt)
runner.callbacks.append(MyCallback())

In [13]:
mtt.reset()
runner.run_one(15, 1)

['measurement_not_associated: 218.74558383443434']
['initialize_new_track: 434.60140624346457', 'measurement_not_associated: 218.74558383443434']
track initialized 1 434.60140624346457
['associate_to_track: 534.2658887074845', 'track_not_updated: 434.60140624346457', 'measurement_not_associated: 218.74558383443434']
track updated 1 534.2658887074845
['associate_to_track: 720.0575261029851', 'track_not_updated: 534.2658887074845', 'measurement_not_associated: 218.74558383443434']
track updated 1 720.0575261029851
['associate_to_track: 911.5846923238271', 'track_not_updated: 720.0575261029851', 'measurement_not_associated: 218.74558383443434']
track updated 1 911.5846923238271
['associate_to_track: 1104.0550923351882', 'track_not_updated: 911.5846923238271', 'measurement_not_associated: 218.74558383443434']
track updated 1 1104.0550923351882
['associate_to_track: 1295.439024622281', 'track_not_updated: 1104.0550923351882', 'measurement_not_associated: 218.74558383443434']
track updated 1

In [11]:
runner.callback.one_x_hat

AttributeError: 'TrackerRunner' object has no attribute 'callback'